In [1]:
PATH = '/home/g056122/FeatExt_Data_Clustering/final_github/content_based_filtering'

In [2]:
DATAPATH = '/home/g056122/ISG_studienarbeit/Movielens_100K'

In [3]:
# import required libraries
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import mean_squared_error,mean_absolute_error
from validate import evaluation
from valres import result 
import pickle

In [4]:
# import movies dataset
dfmovies = pd.read_csv("/home/g056122/ISG_studienarbeit/Movielens_100K/u.item", sep='|', header=None, engine='python', encoding='latin-1')
dfmovies.columns = ['movieId', 'movietitle', 'releasedate', 'videoreleasedate','imdburl', 'unknown', 'action', 'adventure', 'animation','childrens', 'comedy', 'crime', 'documentary', 'drama', 'fantasy', 'film-Noir', 'horror', 'musical', 'mystery', 'romance', 'sci-fi','thriller', 'war', 'western']
copydfmovies = dfmovies.copy()

In [5]:
# drop videoreleasedate and imdburl feature
dfmovies = dfmovies.drop(['movietitle','videoreleasedate', 'imdburl'], axis=1)

In [6]:
# release date feature transformation
dfmovies['releasedate'] = pd.to_datetime (dfmovies['releasedate'], format='%d-%b-%Y')
dfmovies['year']=dfmovies['releasedate'].dt.year
dfmovies = dfmovies.drop('releasedate', axis=1)

In [7]:
dfmovies.head()

,movieId,unknown,action,adventure,animation,childrens,comedy,crime,documentary,drama,...,film-Noir,horror,musical,mystery,romance,sci-fi,thriller,war,western,year
0,1,0,0,0,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1995.0
1,2,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1995.0
2,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1995.0
3,4,0,1,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,1995.0
4,5,0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,1,0,0,1995.0


In [8]:
# encode year feature in dataset
dfmovies = pd.get_dummies(dfmovies, columns = ['year'])

In [9]:
dfmovies.describe()

,movieId,unknown,action,adventure,animation,childrens,comedy,crime,documentary,drama,...,year_1989.0,year_1990.0,year_1991.0,year_1992.0,year_1993.0,year_1994.0,year_1995.0,year_1996.0,year_1997.0,year_1998.0
count,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,...,1682.000000,1682.000000,1682.00000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000
mean,841.500000,0.001189,0.149227,0.080262,0.024970,0.072533,0.300238,0.064804,0.029727,0.431034,...,0.008918,0.014269,0.01308,0.021998,0.074911,0.127229,0.130202,0.211058,0.170036,0.038644
std,485.695893,0.034473,0.356418,0.271779,0.156081,0.259445,0.458498,0.246253,0.169882,0.495368,...,0.094041,0.118632,0.11365,0.146719,0.263326,0.333329,0.336626,0.408181,0.375776,0.192803
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,421.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,841.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1261.750000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1682.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [10]:
# import ratings dataset
dfratings = pd.read_csv(f"{DATAPATH}/u.data", sep='\t', header=None, engine='python', encoding='latin-1')
dfratings.columns = ['userId', 'movieId', 'rating', 'timestamp']

In [11]:
# transform timestamp dataset
dfratings['timestamp'] = pd.to_datetime (dfratings['timestamp'], unit='s')
dfratings['year']=dfratings['timestamp'].dt.year

In [12]:
# scalerize ratings dataset
scaler = MinMaxScaler()
dfratings[['year', 'rating']]=scaler.fit_transform(dfratings[['year', 'rating']])
dfratings = dfratings.drop('timestamp', axis=1)

In [13]:
dfratings.describe()

,userId,movieId,rating,year
count,100000.00000,100000.000000,100000.000000,100000.000000
mean,462.48475,425.530130,0.632465,0.471010
std,266.61442,330.798356,0.281418,0.499161
min,1.00000,1.000000,0.000000,0.000000
25%,254.00000,175.000000,0.500000,0.000000
50%,447.00000,322.000000,0.750000,0.000000
75%,682.00000,631.000000,0.750000,1.000000
max,943.00000,1682.000000,1.000000,1.000000


In [14]:
# merge movies and ratings dataset
dfratings = pd.merge(dfmovies, dfratings, on='movieId')

In [15]:
dfratings.describe()

,movieId,unknown,action,adventure,animation,childrens,comedy,crime,documentary,drama,...,year_1992.0,year_1993.0,year_1994.0,year_1995.0,year_1996.0,year_1997.0,year_1998.0,userId,rating,year
count,100000.000000,100000.0000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,...,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.00000,100000.000000,100000.000000
mean,425.530130,0.0001,0.255890,0.137530,0.036050,0.071820,0.298320,0.080550,0.007580,0.398950,...,0.023130,0.066570,0.088930,0.104990,0.197080,0.173930,0.009910,462.48475,0.632465,0.471010
std,330.798356,0.0100,0.436362,0.344408,0.186416,0.258191,0.457523,0.272144,0.086733,0.489685,...,0.150317,0.249277,0.284644,0.306542,0.397795,0.379051,0.099055,266.61442,0.281418,0.499161
min,1.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.00000,0.000000,0.000000
25%,175.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,254.00000,0.500000,0.000000
50%,322.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,447.00000,0.750000,0.000000
75%,631.000000,0.0000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,682.00000,0.750000,1.000000
max,1682.000000,1.0000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,943.00000,1.000000,1.000000


In [16]:
# split the dataset to X(features) and y(labels)
X = dfratings.drop('rating', axis=1).values
y = dfratings['rating'].values

In [17]:
# kfold cross validation
K = evaluation( X, y, 5, 100)
cv = K.kfold()

Fold 1:
Fold 2:
Fold 3:
Fold 4:
Fold 5:


In [18]:
#split tuple 
classes, eucdis = cv
classes, eucdis

(array([[3.33, 3.26, 3.35, ..., 2.87, 2.32, 2.22],
        [4.42, 3.78, 4.03, ..., 2.85, 2.69, 2.95],
        [3.27, 3.92, 4.56, ..., 1.73, 2.85, 2.05],
        [4.32, 3.23, 4.85, ..., 2.04, 1.97, 3.38],
        [4.64, 3.07, 3.8 , ..., 2.66, 1.98, 2.21]]),
 array([[0.75, 0.75, 0.75, ..., 0.25, 0.  , 0.5 ],
        [0.75, 0.5 , 1.  , ..., 0.25, 0.5 , 0.5 ],
        [0.5 , 0.5 , 0.75, ..., 0.25, 0.75, 0.25],
        [1.  , 0.75, 0.5 , ..., 0.5 , 0.  , 0.5 ],
        [0.75, 0.5 , 0.75, ..., 0.5 , 0.75, 0.5 ]]))

In [19]:
# metrics
ans = result(classes, eucdis)
Metrics = ans.validate()

Metric | Mean | Standard Deviation
RMSE 2.966344472675395, 0.0045146902588294666
MAE 2.8924875000000005, 0.004444126235830993
